In [15]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [16]:
# Importation du dataset et creation du dataset

etudiants = pd.read_csv("TB_ETUDIANT.csv", encoding='latin-1', delimiter=';')

df = pd.DataFrame(etudiants)

df.head()

,ID_ETUDIANT,Sexe,MoyenneBac,NotesMaths,NotesFrançais,NotesPhysChimie,NotesSVT,NotesAnglais,Interets1,Interets2,Orientation,Licence
0,ET001,F,15.0,16.0,16.0,12.75,8.0,6.0,agriculture,agro-alimentaire,agronomie,oui
1,ET002,F,10.0,7.0,15.0,20.00,20.0,13.0,Chimie,sociologie,Biologie,oui
2,ET003,F,10.0,14.0,17.0,17.00,17.0,9.0,Architecture,Mathematiques,Genie civil,oui
3,ET004,F,15.0,7.0,16.0,12.75,8.0,19.0,Informatique,Multimedia,Informatique,non
4,ET005,F,12.0,9.0,13.0,19.00,19.0,15.0,sociologie,Social,Medecine,oui


In [17]:
print('Number of rows : ', len(df))
print('---------------------')
# Count the missing values
print('Missing values count : \n', df.isnull().sum())

Number of rows :  1147
---------------------
Missing values count : 
 ID_ETUDIANT        0
Sexe               0
MoyenneBac         0
NotesMaths         0
NotesFrançais      0
NotesPhysChimie    0
NotesSVT           0
NotesAnglais       0
Interets1          0
Interets2          0
Orientation        0
Licence            0
dtype: int64


In [18]:
le_interets = LabelEncoder()
le_interets.fit(df['Interets1'])
le_interets.fit(df['Interets2'])

df['Interets2'] = le_interets.fit_transform(df['Interets2'])
df['Interets1'] = le_interets.fit_transform(df['Interets1'])


# Séparation des variables indépendantes (X) et de la variable cible (y)
X = df[['MoyenneBac', 'NotesMaths', 'NotesFrançais', 'NotesPhysChimie', 'NotesSVT', 'NotesAnglais',  'Interets1', 'Interets2']]
y = df['Licence']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)



In [19]:
categorical_cols = ['Interets1', 'Interets2']


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)


decision_tree_model = DecisionTreeClassifier()


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', decision_tree_model)
])


pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Interets1',
                                                   'Interets2'])])),
                ('classifier', DecisionTreeClassifier())])

Entrainement et évaluation

In [20]:
classifiers = {
    'Logistic Regression' : LogisticRegression(),
    'decision tree' : DecisionTreeClassifier(),
    'KNN' : KNeighborsClassifier()
}


def train_and_evaluation(classifier):
    for index, (name, classifier) in enumerate(classifier.items()):
        classifier.fit(X_train, y_train)
        y_pred_train = classifier.predict(X_train)
        y_pred_test = classifier.predict(X_test)

        print(f"{name} \n")
        print(f"\t Accuracy (train) : {accuracy_score(y_train, y_pred_train):.2f}")
        print(f"\tAccuracy (test) : {accuracy_score(y_test, y_pred_test):.2f}")
        print(f"{classification_report(y_test, y_pred_test, zero_division=1)}\n")




In [21]:
train_and_evaluation(classifiers)

Logistic Regression 

	 Accuracy (train) : 0.60
	Accuracy (test) : 0.61
              precision    recall  f1-score   support

         non       0.00      0.00      0.00       133
         oui       0.61      0.99      0.75       212

    accuracy                           0.61       345
   macro avg       0.31      0.49      0.38       345
weighted avg       0.38      0.61      0.46       345


decision tree 

	 Accuracy (train) : 1.00
	Accuracy (test) : 0.53
              precision    recall  f1-score   support

         non       0.38      0.36      0.37       133
         oui       0.61      0.63      0.62       212

    accuracy                           0.53       345
   macro avg       0.50      0.50      0.50       345
weighted avg       0.52      0.53      0.52       345


KNN 

	 Accuracy (train) : 0.71
	Accuracy (test) : 0.56
              precision    recall  f1-score   support

         non       0.41      0.35      0.38       133
         oui       0.63      0.68      0.

In [22]:
# Nouvel étudiant pour la prédiction
nouvel_etudiant = {
     'MoyenneBac' : 14,
    'NotesMaths': 12,
    'NotesFrançais': 7,
    'NotesPhysChimie': 12,
    'NotesSVT': 10,
    'NotesAnglais': 8,
    'Interets1': 'Physiques',  
    'Interets2': 'Informatique' 
}


# Création d'un DataFrame pour le nouvel étudiant
df = pd.DataFrame([nouvel_etudiant])

# Prédiction pour le nouvel étudiant
prediction = pipeline.predict(df)

# Affichage de la prédiction pour le nouvel étudiant
print("Recommandations d'orientation pour le nouvel étudiant :")
print(f"Prédiction : {prediction}")

Recommandations d'orientation pour le nouvel étudiant :
Prédiction : ['oui']
